In [2]:
import pandas as pd
import cv2
import os
import shutil

In [7]:
ori_df = pd.read_csv(r"E:\claassifier\images_3rd" + '/result.csv')
ori_df.sum()

id        None_20230710090535_2_L_1.jpgNone_202307100905...
good                                                     74
bubble                                                    0
edge                                                    353
dtype: object

In [8]:
add_df = pd.read_csv('csvs/dataset_sec_add.csv')
add_df = add_df[add_df['good'] == 1]

random_rows = add_df.sample(n=280)
random_rows

,id,good,defect,no_lens
7488,20230825162204_2_L_1.jpg,1,0,0
6271,lr20230817142402_2_R_1.jpg,1,0,0
3478,20230731171434_0_L_1.jpg,1,0,0
5716,20230817121637_1_R_1.jpg,1,0,0
852,20230721155229_0_L_1.jpg,1,0,0
...,...,...,...,...
1753,lr20230725102544_3_L_1.jpg,1,0,0
6471,lr20230822094254_4_R_1.jpg,1,0,0
492,20230721141230_0_R_1.jpg,1,0,0
3412,20230731145713_3_R_1.jpg,1,0,0


In [13]:
check_dir = r'E:\claassifier\images_sec_full'
false_count = 0
for filenames in random_rows['id']:
    if os.path.isfile(check_dir + '/' + filenames):
        shutil.copy(check_dir + '/' + filenames, 'data/polar/' + filenames)
    else:
        false_count += 1
false_count

0

In [25]:
df = pd.concat([ori_df, random_rows])
df['defect'] = 0
df.fillna(0, inplace=True)  # NaN 값을 0으로 치환
df['defect'] = ((df['bubble'] != 0) | (df['edge'] != 0) | (df['no_lens'] != 0)).astype(int)

df = df.drop(columns=['bubble', 'edge', 'no_lens'])
df[df['good'] == 0].sum()

id        None_20230710090535_2_L_1.jpgNone_202307100905...
good                                                      0
defect                                                  353
dtype: object

In [3]:
#df.to_csv('csvs/polar.csv', index=False)
df = pd.read_csv('csvs/polar.csv')
df

,id,good,defect
0,None_20230710090535_2_L_1.jpg,0,1
1,None_20230710090535_3_L_1.jpg,0,1
2,None_20230710090535_4_L_1.jpg,0,1
3,None_20230710090654_0_L_1.jpg,0,1
4,None_20230710090654_1_L_1.jpg,0,1
...,...,...,...
702,lr20230725102544_3_L_1.jpg,1,0
703,lr20230822094254_4_R_1.jpg,1,0
704,20230721141230_0_R_1.jpg,1,0
705,20230731145713_3_R_1.jpg,1,0


In [19]:
############### 1C 모델에서 burr, b_edge, broken을 불러옴.
new_df_ori = pd.read_csv('csvs/dataset.csv')
new_df = new_df_ori[(new_df_ori['b_edge'] == 1) | (new_df_ori['burr'] == 1) | (new_df_ori['borken'] == 1)]
new_dataset = [
    {'id': new_df.id, 'good': 0, 'defect': 1},
    {'id': new_df_ori[new_df_ori['good'] == 1].id, 'good': 1, 'defect': 0}
]
for data in new_dataset:
    temp_df = pd.DataFrame(data)  # 각 데이터셋을 DataFrame으로 변환
    df = pd.concat([df, temp_df], ignore_index=True)
df

,id,good,defect
0,None_20230710090535_2_L_1.jpg,0,1
1,None_20230710090535_3_L_1.jpg,0,1
2,None_20230710090535_4_L_1.jpg,0,1
3,None_20230710090654_0_L_1.jpg,0,1
4,None_20230710090654_1_L_1.jpg,0,1
...,...,...,...
10853,None_20230508115324_0_R_1.jpg,1,0
10854,None_20230508115340_1_L_1.jpg,1,0
10855,None_20230508115554_1_L_1.jpg,1,0
10856,None_20230508115716_2_R_1.jpg,1,0


In [21]:
############### 1d, 2d 추가 데이터셋에서 불러옴.

directory_path = "e:/polar_data/"
df_2d = pd.read_csv( directory_path + '2D.csv')
df_1d = pd.read_csv( directory_path + '1D.csv')

df_1d_good = df_1d[df_1d['good'] == 1]
df_2d_good = df_2d[df_2d['good'] == 1]

df_2d_edge = df_2d[df_2d['edge'] == 1]
df_1d_edge = df_1d[df_1d['edge'] == 1]

df_1d2d_edge = pd.concat([df_1d_edge, df_2d_edge], ignore_index=True)
new_dataset = [
    {'id': df_1d2d_edge.id, 'good': 0, 'defect': 1},
    {'id': df_1d_good.sample(n=14000, random_state=42).id, 'good': 1, 'defect': 0},
    {'id': df_2d_good.sample(n=14000, random_state=42).id, 'good': 1, 'defect': 0}
]
for data in new_dataset:
    temp_df = pd.DataFrame(data)  # 각 데이터셋을 DataFrame으로 변환
    df = pd.concat([df, temp_df], ignore_index=True)
df

,id,good,defect
0,None_20230710090535_2_L_1.jpg,0,1
1,None_20230710090535_3_L_1.jpg,0,1
2,None_20230710090535_4_L_1.jpg,0,1
3,None_20230710090654_0_L_1.jpg,0,1
4,None_20230710090654_1_L_1.jpg,0,1
...,...,...,...
67803,240320-08.6.7-B003_20240321194821_0_L_1.jpg,1,0
67804,240322-05.15.16-B002_20240326213735_0_L_1.jpg,1,0
67805,240513-04.10.15-A002_20240514200910_3_R_1.jpg,1,0
67806,240223-02.5.6-B002_20240226213617_3_R_1.jpg,1,0


In [31]:
import tqdm
for rows in tqdm.tqdm(df_1d_good.sample(n=14000, random_state=42).itertuples(), total=len(df_1d_good.sample(n=14000, random_state=42))):
    if os.path.isfile('data/polar/' + rows.id):
        continue
    else:
        try:
            shutil.copy(directory_path + '1D/' + rows.id, 'data/polar/' + rows.id)
        except:
            print(rows.id)
            continue


100%|██████████| 14000/14000 [00:00<00:00, 34912.74it/s]


In [32]:
for rows in tqdm.tqdm(df_2d_good.sample(n=14000, random_state=42).itertuples(), total=len(df_2d_good.sample(n=14000, random_state=42))):
    if os.path.isfile('data/polar/' + rows.id):
        continue
    else:
        try:
            shutil.copy(directory_path + '2D/' + rows.id, 'data/polar/' + rows.id)
        except:
            print(rows.id)
            continue


 60%|█████▉    | 8364/14000 [00:00<00:00, 27799.65it/s]

#NAME?


100%|██████████| 14000/14000 [00:00<00:00, 27833.04it/s]

#NAME?


In [33]:

for rows in tqdm.tqdm(df_1d_edge.itertuples(), total=len(df_1d_edge)):
    if os.path.isfile('data/polar/' + rows.id):
        continue
    else:
        try:
            shutil.copy(directory_path + '1D/' + rows.id, 'data/polar/' + rows.id)
        except:
            print(rows.id)
            continue


100%|██████████| 19590/19590 [00:00<00:00, 50102.54it/s]

#NAME?
#NAME?
#NAME?


In [34]:
for rows in tqdm.tqdm(df_2d_edge.itertuples(), total=len(df_2d_edge)):
    if os.path.isfile('data/polar/' + rows.id):
        continue
    else:
        try:
            shutil.copy(directory_path + '2D/' + rows.id, 'data/polar/' + rows.id)
        except:
            print(rows.id)
            continue


100%|██████████| 9360/9360 [00:00<00:00, 49523.96it/s]

#NAME?


In [41]:
cccc = 0
for rows in tqdm.tqdm(new_df.itertuples(), total=len(new_df)):
    if os.path.isfile('data/polar/' + rows.id):
        continue
    else:
        try:
            shutil.copy('data/images2/' + rows.id, 'data/polar/' + rows.id)
        except:
            cccc += 1
            continue
cccc

100%|██████████| 5204/5204 [02:40<00:00, 32.33it/s]


0

In [36]:
for rows in tqdm.tqdm(new_df_ori[new_df_ori['good'] == 1].itertuples(), total=len(new_df_ori[new_df_ori['good'] == 1])):
    if os.path.isfile('data/polar/' + rows.id):
        continue
    else:
        try:
            shutil.copy('E:/claassifier/images_firstmodel/' + rows.id, 'data/polar/' + rows.id)
        except:
            print(rows.id)
            continue

100%|██████████| 4947/4947 [00:00<00:00, 47567.81it/s]


In [24]:
df.sum()

id        None_20230710090535_2_L_1.jpgNone_202307100905...
good                                                  33301
defect                                                34507
dtype: object

In [25]:
df.to_csv('csvs/polar_60000.csv', index=False)